# IMT 574 Problem Set 7

Loading libraries

In [1]:
#IPython is what you are using now to run the notebook
import IPython
print( "IPython version:      %6.6s (need at least 1.0)" % IPython.__version__)

# Numpy is a library for working with arrays and matrices
import numpy as np
print( "Numpy version:        %6.6s (need at least 1.7.1)" % np.__version__)

# SciPy implements many different numerical algorithms
import scipy as sp
print( "SciPy version:        %6.6s (need at least 0.12.0)" % sp.__version__)

# Pandas makes working with data tables easier
import pandas as pd
print( "Pandas version:       %6.6s (need at least 0.11.0)" % pd.__version__)

# Module for plotting
import matplotlib.pyplot as plt  
from pylab import *
print( "Matpltolib version:    %6.6s (need at least 1.2.1)" %
       matplotlib.__version__)
%matplotlib inline
# necessary for in-line graphics

# SciKit Learn implements several Machine Learning algorithms
import sklearn
print("Scikit-Learn version: %6.6s (need at least 0.13.1)" %
       sklearn.__version__)
import os
# for certain system-related functions

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

IPython version:       7.8.0 (need at least 1.0)
Numpy version:        1.16.5 (need at least 1.7.1)
SciPy version:         1.3.1 (need at least 0.12.0)
Pandas version:       0.25.1 (need at least 0.11.0)
Matpltolib version:     3.1.1 (need at least 1.2.1)
Scikit-Learn version: 0.21.3 (need at least 0.13.1)


## 1 Explore and clean the data

First, let's load data and take a closer look at it.

In [2]:
# Reading the world Value data
amzrev_df = pd.read_csv("amazon-reviews.csv.bz2", sep ='\t')

In [3]:
print(amzrev_df.shape)
amzrev_df.head(10)

(205331, 4)


,date,summary,review,rating
0,2013-07-16,Awesine,Perfect for new parents. We were able to keep ...,5
1,2013-06-29,Should be required for all new parents!,This book is such a life saver. It has been s...,5
2,2014-03-19,Grandmother watching baby,Helps me know exactly how my babies day has go...,5
3,2013-08-17,repeat buyer,I bought this a few times for my older son and...,5
4,2014-04-01,Great,I wanted an alternative to printing out daily ...,4
5,2014-05-10,"Great for basics, but not detail","This is great for basics, but I wish the space...",4
6,2013-07-17,Perfect for the working mom,My 3 month old son spend half of his days with...,5
7,2013-01-27,Great for newborns,This book is perfect! I'm a first time new mo...,5
8,2014-04-22,"It's ok, but I liked a regular weekly planner ...","I wanted to love this, but it was pretty expen...",3
9,2013-11-19,Best for Tracking!,The Baby Tracker brand books are the absolute ...,5


Checking the number of missing values present in all columns.

In [4]:
amzrev_df.isna().sum()

date        0
summary    15
review     80
rating      0
dtype: int64

1. Load the data. Remove all the missing and empty observations of review.

In [5]:
amzrev_df=amzrev_df.dropna(subset=["review"])
amzrev_df.shape

(205251, 4)

2. You almost certainly need to take a subset unless you run this on a beefy server. I'd recommend to start with 1000 reviews and scale it up to 10 or 20k when the things seem to work well (requires 2-4G of RAM). Note: please take a random sample, not just first n lines. If you want to make your results replicable, use np.random.seed.

In [6]:
np.random.seed(1)
amzrev_df_sample=amzrev_df.sample(10000) #Taking a sample of 10k observations
amzrev_df_sample=pd.DataFrame(amzrev_df_sample)
amzrev_df_sample

,date,summary,review,rating
187169,2016-09-04,"Pricey, but with some benefits",3.5 starts - First thing that hits you is the ...,4
145513,2014-01-04,Couldn't survive without it.,Life will never be the same again with baby in...,5
150682,2013-09-06,Your baby as a mini Joe Namath with a horse sh...,I always see things like this and wonder why d...,4
185568,2015-10-21,"La Roche-Posay New Pigment Serum, stands up to...",One of the most reputable names in skincare is...,5
122306,2013-10-31,For a limited time only,The one thing I did like about this monitor is...,1
...,...,...,...,...
68195,2014-02-27,Great,I was concerned this wouldn't fit well because...,5
201096,2011-06-19,Nice amp for the money,Nice amp for the money but couldn't push a sin...,3
69363,2012-01-03,Great bibs,We had bought a couple of these bibs when our ...,5
5256,2013-02-27,Love,Couldn't live without this pillow. I was a sto...,5


3. Take a look some of the reviews. Just be looking at those, imagine how you might cluster those into a few categories.

Answer:

By looking at the reviews, we can see that most reviews have words pertaining to various products such as instruments, baby, skincare, baby clothes and so on. Thus, we can cluster reviews based on the categories of the product they include for example: baby product reviews, skincare and beauty product reviews, musical instrument product reviews and so on.

## 2 Implement TF-IDF transform

1. Transform a subset (1000 or so) reviews into BOW using sklearns CountVectorizer. Ensure you get the actual count of words, not just binary presence/absence.

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(amzrev_df_sample.review.values)
words = vectorizer.get_feature_names()

In [8]:
bow_df = pd.DataFrame.sparse.from_spmatrix(X)
bow_df.columns = words
bow_array = X.toarray()
sum_array = list(bow_array.sum(axis = 0))
count_main = dict(zip(words, sum_array))
bow_df

,00,000,008,01,03,03oz,04,05,06,06my,...,zooming,zooper,zopper,zorb,zoya,zulily,zumba,zutano,zwilling,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. Transform the BOW into tf-idf matrix. TF-IDF is a way to weigh the frequency and importance of the words, so common words get low weight and specific words, present in a subset of documents get a high weight.

In [9]:
tf_df = np.log( 1 + bow_df)

In [10]:
N = bow_df.shape[0]

#Finding frequency of words
frequency_words = (bow_df > 0).sum(axis = 0)

#Finding idf {j} matrix on our Bag Of Words
idf = np.log(N / ( 1 + frequency_words ))

In [11]:
tfidf = tf_df * idf

In [12]:
tfidf

,00,000,008,01,03,03oz,04,05,06,06my,...,zooming,zooper,zopper,zorb,zoya,zulily,zumba,zutano,zwilling,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3 Implement k-Means Clustering

1. Pick k random reviews (remember: we work on TF-IDF matrix, so we talk about picking their TF-IDF) and assign those to be the cluster centers.

In [13]:
#picking k=3
k=3
np.random.seed(0)
centers = tfidf.sample(k)

2. Assign all reviews into arbitrary clusters (all to the first is fine).

In [14]:
clusters_initial = np.zeros(tfidf.shape[0])

3. Compute the norm of all review vectors
4. Compute the norm of the cluster center vectors
5. For each cluster, compute the cosine similarity between the reviews and the corresponding cluster centers.

In [15]:
def get_cossim(reviews , centers):
    dot_product = np.dot(reviews, centers.T)
    review_norm = np.linalg.norm(reviews, axis =1)
    review_norm = review_norm.reshape(reviews.shape[0],1)
    center_norm = np.linalg.norm(centers, axis = 1)
    center_norm = center_norm.reshape(centers.shape[0],1)
    return dot_product / (review_norm.dot(center_norm.T))

In [16]:
similarity = get_cossim(tfidf, centers)

6. Now assign each review to the cluster, center of which is the closest to it in terms of cosine similarity.

In [17]:
review_clusters= np.array([np.where(similarity[i]==min(similarity[i]))[0][0] for i in range(similarity.shape[0])])
review_clusters[1:15]

array([0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0], dtype=int64)

7. Did any of the reviews change cluster? If no, then you are done.

In [18]:
if any(review_clusters - clusters_initial != 0):
    print("yes")

yes


8. Compute new cluster centers by taking mean (centroid) of all reviews that belong to each cluster.

In [19]:
tfidf=np.array(tfidf)
new_centers = [np.mean(tfidf[np.where(np.array(review_clusters)==i)], axis=0) for i in range(k)]

In [20]:
new_centers

[array([0.01290811, 0.00270534, 0.00079297, ..., 0.00079297, 0.00195222,
        0.00079297]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.01103721, 0.00664582, 0.        , ..., 0.        , 0.        ,
        0.        ])]

9. Repeat from 4. In my experience, you need approx 30 iterations, more for more clusters.

In [21]:
k = 3
centers_old = tfidf[:k]
clusters_old = np.zeros(tfidf.shape[0])

iters=0

while iters<30:

    #calculate cosine similarity between tfidf matrix and cluster centers 
    # cosine similarity = 1- cosine distance
    similarity= get_cossim(tfidf, centers_old)
    
    #Assigning the reviews to new clusters
    clusters_new= np.array([np.where(similarity[i]==max(similarity[i]))[0][0] for i in range(similarity.shape[0])])

    # check if clusters have changed i.e has algorithm converged?
    if all(clusters_new - clusters_old == 0):
        print("Converged at iteration", iters)
        break
    #calculate new centers
    new_centers = np.array([np.mean(tfidf[np.where(np.array(clusters_new)==i)], axis=0) for i in range(k)])
    
    # store old clusters and old centers for next iteration
    clusters_old = clusters_new
    centers_old = new_centers
    
    # increment iterator
    iters=iters+1

Converged at iteration 11


Question: What kind of properties of the reviews is the algorithm considering? What constitutes similar reviews?

Answer:

The algorithm considers the tf-idf weights for each word present in a review while clustering the reviews. The tf-idf score reflects how the importance of a word in a review belonging to a collection of reviews. Thus our algorithm will group the reviews having a similar tf-idf scores for important words into distinct clusters.

## 4 Play and Analyze Your Clusters

1. Split the (subset of) reviews into clusters
2. Print out a set of reviews from each cluster, so you can see and compare the results.

In [22]:
# Printing 10 reviews from cluster 0
cluster0_reviews = amzrev_df.iloc[np.where(clusters_new==0)]['review'][1:10]
for i in cluster0_reviews.index:
    print("Review: ",cluster0_reviews[i])
    print("\n")

Review:  I bought this a few times for my older son and have bought it again for my newborn. This is super easy to use and helps me keep track of his daily routine. When he started going to the sitter when I went back to work, it helped me know how his day went to better prepare me for how the evening would most likely go. When he was sick, it help me keep track of how many diapers a day he was producing to make sure he was getting dehydrated. The note sections to the side and bottom are useful too because his sitter writes in small notes about whether or not he liked his lunch or if the playtime included going for a walk, etc.Excellent for moms who are wanting to keep track of their kids daily routine even though they are at work. Excellent for dads to keep track as my husband can quickly forget what time he fed our son. LOL


Review:  This book is a great way for keeping track of daily changing and feedings.  My only wish is that it was in a 24 hour format.  But other than that it is

In [23]:
# Printing 10 reviews from cluster 1
cluster1_reviews = amzrev_df.iloc[np.where(clusters_new==1)]['review'][1:10]
for i in cluster1_reviews.index:
    print("Review: ",cluster1_reviews[i])
    print("\n")

Review:  I use this so that our babysitter (grandma) can keep track of what goes on during the day.  We've recorded so many milestones since we received this.  It is a 6 month diary and has room for notes.  I wish the hours started around 6 am and went until 8 pm, but we just write over certain times to suit our purposes. The nice thing is that if we had a rough night and we are super tired, we have something to refer to if we forgot when we last fed the baby or when he woke up. I also like that the front page has a place for emergency comments and a consent form in case your child needs medical care.  It also has a place at the back of the book for immunizations.  I'll definitely get another one in 6 months when we finish this book.


Review:  Easy to use, simple! I got this when my baby was 2 months old and love it. It makes it easy to track things and write down developmental notes in an organized way. I know moms who create spreadsheets of their baby's nap times, etc. But I'm not o

In [24]:
# Printing 10 reviews from cluster 2
cluster2_reviews = amzrev_df.iloc[np.where(clusters_new==2)]['review'][1:10]
for i in cluster2_reviews.index:
    print("Review: ",cluster2_reviews[i])
    print("\n")

Review:  I wanted an alternative to printing out daily log sheets for the nanny to fill out, and this has worked out great! I'm no longer searching my daughter's bag for a crumpled piece of paper each day. It's also nice to be able to look back on previous days and weeks for eating and sleeping patterns. I would have preferred a plastic-type cover, but it's held up well so far.


Review:  This is great for basics, but I wish the space to write things in was bigger. A lot times I need struggle trying to read what the caretaker wrote in because the spaces go together.


Review:  My 3 month old son spend half of his days with my mother and half with a neighbor while I worked.  I had them track his activity (loosely) in this which allowed me to get an idea of how his schedule was developing and how much milk he was eating.  It was the best way to have some cohesion in his life while I was at work.


Review:  This book is perfect!  I'm a first time new mom, and this book made it so easy to 

3. Comment on what do you see. How good a job does the algorithm in picking up meaningful similarities between reviews?

By looking at the reviews from the above 3 clusters we can see that most reviews are related to baby and baby products, with an odd review pertaining to a life goals planner and books. The algorithm has not done a very good job in picking up meaningful similarities between reviews as there is not much difference in the categories of reviews belonging to different clusters.

4. Play with a few different k values. If your code works well, now it is time to scale up to as large sample as you can (I can do 30k in approx 5 mins on an oldish desktop).

In [25]:
#Using k=16 as it gave me better defined clusters.

k = 16
centers_old = tfidf[:k]
clusters_old = np.zeros(tfidf.shape[0])

iters=0

while iters<100:
    
    #calculate cosine similarity between tfidf matrix and cluster centers 
    # cosine similarity = 1- cosine distance
    similarity= get_cossim(tfidf, centers_old)
    
    #Assigning the reviews to new clusters
    clusters_new= np.array([np.where(similarity[i]==max(similarity[i]))[0][0] for i in range(similarity.shape[0])])

    # check if clusters have changed i.e has algorithm converged?
    if all(clusters_new - clusters_old == 0):
        print("Converged at iteration", iters)
        break
    #calculate new centers
    new_centers = np.array([np.mean(tfidf[np.where(np.array(clusters_new)==i)], axis=0) for i in range(k)])
    
    # store old clusters and old centers for next iteration
    clusters_old = clusters_new
    centers_old = new_centers
    
    # increment iterator
    iters=iters+1

Converged at iteration 30


5. Give some sort of final comments. How well did it work? How many clearly distinct clusters did you see? Which k was the most interesting?

In [26]:
#Printing 10 reviews from each cluster 
for i in range(16):
    cluster_reviews = amzrev_df.iloc[np.where(clusters_new==i)]['review'][100:111]
    print("\nThe Reviews of cluster" , i, "are:\n")
    for j in cluster_reviews.index:
        print("Review: ",cluster_reviews[j])
        print("\n")


The Reviews of cluster 0 are:

Review:  This potty is great. As the previous reviewer mentioned, it's one piece so nowhere for the pee to go except inside. I think the only fault is that it's incredibly low to the ground, so although that is great for new learners, once they master the potty training and you just need an extra around the house, it doesn't encourage them to go because it's too short. If you can spend a bit more, the newer model with the inside that comes out ($24.95) is much better in my opinion. HIgher for sure, and actually easier to dump since it has an insert. Make sure never to use bleach in these, though, I ruined the bottom of one of these by letting it sit in a 50% bleach solution, it ate away the smooth bottom.


Review:  We got this one and the Summer Infant and they're so similar you might as well get the cheaper one. This one is definitely taller but our son hasn't had better or worse luck with either one.


Review:  This is a great potty seat and very easy

Answer:

On changing k a few times we found that k=16 is giving us better defined clusters. We can see that for many clusters, reviews belonging to various categories such as potty seats, baby bottles, bath tubs, toys, playpens, diaper pails, baby mattresses, teething toys, baby crawlers are grouped into distinct clusters. However, there are still many clusters containing reviews which do not lend themselves to any particular category. Also, I could not identify clusters belonging to skincare or musical instruments distinctly. All-in-all, the k-means clustering algorithm using tf-idf did an average job in clustering the reviews.